In [1]:
# Importing Libraries
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import interact, widgets, Button, HBox, VBox, Output
from IPython.display import display
import vtk
from vtk.util import numpy_support

In [2]:
# Function to load the Dataset
def load_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    data = reader.GetOutput()
    dims = data.GetDimensions()
    scalar_range = data.GetScalarRange()
    array = data.GetPointData().GetScalars()
    numpy_array = numpy_support.vtk_to_numpy(array).reshape(dims, order='F')
    return numpy_array, scalar_range

# Loading the 3D dataset
dataset, scalar_range = load_vti("mixture.vti")

In [3]:
# Setting initial values
initial_isovalue = 0.0
initial_histogram_range = scalar_range

In [4]:
# Updating data based on dataset structure
x, y, z = np.meshgrid(np.arange(dataset.shape[0]), np.arange(dataset.shape[1]), np.arange(dataset.shape[2]))

In [5]:
# Defining custom color map
fixed_colorscale = [
    [0.0, 'purple'],
    [0.25, 'indigo'],
    [0.5, 'red'],
    [0.75, 'orange'],
    [1.0, 'yellow']
]

In [6]:
# Creating Isosurface Plotly figure
def create_isosurface(isovalue):
        # Creating Isosurface plot
        fig_isosurface = go.Figure(data=go.Isosurface(
            x=y.flatten(),
            y=x.flatten(),
            z=z.flatten(),
            value=dataset.flatten(),  # Ensure dataset contains scalar values
            isomin=isovalue,
            isomax=isovalue,
            surface_count=1,
            cmin = -1.0,
            cmax = 0.45,
            colorscale=fixed_colorscale,
            showscale=False,  # Remove the color map from display
            opacity=0.9,
            caps=dict(x_show=False, y_show=False, z_show=False)
        ))
        

        # Remove numerical values from axis
        fig_isosurface.update_layout(
            scene=dict(
                xaxis_visible=True,
                yaxis_visible=True,
                zaxis_visible=True,
                xaxis_showticklabels=False,
                yaxis_showticklabels=False,
                zaxis_showticklabels=False
            )
        )
        
        # Setting dimensions of the plot
        fig_isosurface.update_layout(width=500, height=500)
        
        # Showing the figure
        with output:
            output.clear_output(wait=True) # Clearing previous plot
            display(fig_isosurface)


In [7]:
# Creating initial Histogram Plotly figure
def create_initial_histogram(histogram_range):
    data = dataset.flatten()
    
    fig_histogram = px.histogram(x=data, nbins=50, range_x=histogram_range)
    
    fig_histogram.update_layout(
        xaxis_title="Vortex scalar values",
        yaxis_title="Frequency"
    )
    
    # Setting dimensions of the plot
    fig_histogram.update_layout(width=400, height=450)
       
    with output_histogram:
        output_histogram.clear_output(wait=True) # Clearing previous plot
        display(fig_histogram)
            
            
            

# Creating Histogram Plotly figure
def create_histogram(histogram_range):
    data = dataset.flatten()
    hist, bins = np.histogram(data, bins=25, range= histogram_range)
    max_y = max(hist)
    fig_histogram = px.histogram(x=data, nbins=100, range_x=histogram_range, range_y = (0,max_y*1.1))
    
    # Label axis
    fig_histogram.update_layout(
        xaxis_title="Vortex scalar values",
        yaxis_title="Frequency"
    )
    
    # Setting dimensions of the plot
    fig_histogram.update_layout(width=400, height=450)

    
    with output_histogram:
        output_histogram.clear_output(wait=True) # Clearing previous plot
        display(fig_histogram)


In [8]:
# Slider callback function
def update_slider(change):
    isovalue = change.new
    histogram_range = (isovalue - 0.25, isovalue + 0.25)
    create_isosurface(isovalue)
    create_histogram(histogram_range)

In [9]:
# Reset button callback function
def reset_plot(button):
    slider.value = initial_isovalue
    histogram_range = scalar_range
    create_isosurface(initial_isovalue)
    create_initial_histogram(histogram_range)

In [10]:
# Creating slider widget
slider = widgets.FloatSlider(
    value=initial_isovalue,
    min=scalar_range[0],
    max=scalar_range[1],
    step=0.1,
    description='Isovalue:',
    continuous_update=False
)
slider.observe(update_slider, names='value')

In [11]:
# Creating reset button widget
reset_button = Button(description="Reset")
reset_button.on_click(reset_plot)


In [12]:
# Creating output widgets for displaying plots
output = Output()
output_histogram = Output()

In [13]:
# Displaying the plots
display(HBox([slider, reset_button]))
display(HBox([output, output_histogram]))
reset_plot(True)